Logs 
- [2025/08/09]     
  Apply VQE to solve portfolio optimization problem

In [1]:
import json

import itertools
import pennylane as qml
import matplotlib.pyplot as plt

from pennylane import numpy as np

Objective 
$$
  \argmin_{y_{\ell, i}} \sum_{\ell \in L} \sum_{i \in \mathbb{K}_\ell} \gamma_{\ell, i} y_{\ell, i}
  + \frac{1}{2}
    \left(\sum_{\ell\in L} \sum_{i \in \mathbb{K}_\ell} \sum_{j \in \mathbb{K}_\ell} 
    (\alpha_{\ell,i} y_{\ell,i}) (\alpha_{\ell, j} y_{\ell,j}) \right)
  + C
$$

$$
  = \argmin_{y_{\ell, i}}
    \sum_{\ell \in L} \sum_{i \in \mathbb{K}_\ell} \gamma_{\ell, i} y_{\ell, i}
  + 
  \frac{1}{2} \left(\sum_{\ell \in L} \sum_{i \in \mathbb{K}_\ell} 
    \alpha_{\ell,i}^2 y_{\ell, i}^2
  + \sum_{\ell \in L} \sum_{i \in \mathbb{K}_\ell} \sum_{j \in \mathbb{K}_\ell \setminus \{i\}}
    2 \alpha_{\ell, i} \alpha_{\ell, j} y_{\ell, i} y_{\ell, j} \right)
  + C
$$

Constraints

`trade_num_upper.limit`
$$
  \left(\sum_{\ell \in L}  \sum_{i \in \mathbb{K}_\ell} y_{\ell, i} \right) \leq M
$$

`limit_sodraw.filterLevel1_sodraw.filterLevel3_#_fund_enriched.pmv`

$$
  K_{\ell}^\text{min} \leq 
    \sum_{i \in \mathbb{K}_{\ell}} \beta_{\ell, i} y_{\ell, i}
    \leq K_{\ell}^\text{max}, \qquad \forall \ell \in L
$$

`limt_sodrtaw.filterLevel1_sodraw.filterLevel3_#_fund_enriched.dxsCtr`

$$
  O_{\ell}^\text{min} \leq 
    \sum_{i \in \mathbb{K}_{\ell}} \varepsilon_{\ell, i} y_{\ell, i}
    \leq O_{\ell}^\text{max}, \qquad \forall \ell \in L
$$

`low_cnstr_cash`

$$
  P^\text{min} \leq \sum_{\ell \in L}  \sum_{i \in \mathbb{K}_\ell}
    \beta_{\ell, i} y_{\ell, i} \leq P^\text{max}
$$

$y_{\ell, i}$ are binary variables

$$
\begin{align*}
  L &= \{1, 2, 3\}  \\
    &= \{\text{Capital Goods}, \text{Insurance}, \text{Transporation}\}
\end{align*}
$$

$$
  \mathbb{K}_{\ell}
  = \begin{cases}
      \begin{aligned}
      \mathbb{K}_1 &= \{1, 2, 3, \ldots, 12\} 
        = \left\{
        \begin{array}{l}
        \texttt{020002BJ9}, \texttt{026874DS3}, \texttt{15135BAW1}, 
        \texttt{21871XAS8}, \texttt{444859BR2}, \\ \texttt{444859BV3}, 
        \texttt{444859BY7}, \texttt{444859CA8}, \texttt{540424AT5}, 
        \texttt{56501RAN6}, \\ \texttt{759351AP4}, \texttt{91324PEJ7} 
        \end{array}
        \right\},
      \end{aligned} \\[16pt]
      \mathbb{K}_2 = \{1, 2, 3, \ldots, 13\}
        = \left\{
        \begin{array}{l}
          \texttt{081437AT2}, \texttt{097023CJ2}, \texttt{14448CBC7}, 
          \texttt{24422EWZ8}, \texttt{24422EXP9}, \\ \texttt{36166NAK9}, 
          \texttt{438516CM6}, \texttt{443201AC2}, \texttt{45687VAB2}, 
          \texttt{539830CD9}, \\ \texttt{75513EAD3}, \texttt{760759BA7}, 
          \texttt{760759BC3}
        \end{array}
        \right\}, \\[16pt]
      \mathbb{K}_3 = \{1, 2, 3, \ldots, 6\}
        = \left\{
        \begin{array}{l}
          \texttt{13645RAD6}, \texttt{13645RBF0}, \texttt{314353AA1}, 
          \texttt{655844CR7}, \texttt{655844CT3}, \\ \texttt{907818FX1}
        \end{array}
        \right\}
    \end{cases}
$$

$$
  M = \texttt{trade\_num\_upper\_limit} = 500
$$

Transform the above constrained problem into unconstrained problem with
Lagrange multipliers

$$
\begin{align*}
&\argmin_{y_{\ell, i}}
    \sum_{\ell \in L} \sum_{i \in \mathbb{K}_\ell} \gamma_{\ell, i} y_{\ell, i}
  + 
  \frac{1}{2} \left(\sum_{\ell \in L} \sum_{i \in \mathbb{K}_\ell} \alpha_i^2 y_i^2
  + \sum_{\ell \in L} \sum_{i \in \mathbb{K}_\ell} \sum_{j \in \mathbb{K}_\ell \setminus \{i\}}
    2 \alpha_i \alpha_j y_i y_j \right) \\
  &\qquad+
    \lambda_1\left(\sum_{\ell \in L}\sum_{i \in \mathbb{K}_\ell}
      y_{\ell, i} - M + s_1^2 
    \right) \\
  &\qquad+ \lambda_2\left( \sum_{i \in \mathbb{K}_\ell}\beta_{1, i} y_{1, i}
      - K_1^\text{min} + s_2^2
    \right)
    + \lambda_3\left( \sum_{i \in \mathbb{K}_\ell}\beta_{2, i} y_{2, i}
      - K_2^\text{min} + s_3^2
    \right)
    + \lambda_4\left( \sum_{i \in \mathbb{K}_\ell}\beta_{3, i} y_{3, i}
      - K_3^\text{min} + s_4^2
    \right) \\
  &\qquad+ \lambda_5\left( \sum_{i \in \mathbb{K}_\ell}\beta_{1, i} y_{1, i}
      - K_1^\text{max} + s_5^2
    \right)
    + \lambda_6\left( \sum_{i \in \mathbb{K}_\ell}\beta_{2, i} y_{2, i}
      - K_2^\text{max} + s_6^2
    \right)
    + \lambda_7\left( \sum_{i \in \mathbb{K}_\ell}\beta_{3, i} y_{3, i}
      - K_3^\text{max} + s_7^2
    \right) \\
  &\qquad+ \lambda_8\left( \sum_{i \in \mathbb{K}_\ell}\beta_{1, i} y_{1, i}
      - O_1^\text{min} + s_8^2
    \right)
    + \lambda_9\left( \sum_{i \in \mathbb{K}_\ell}\beta_{2, i} y_{2, i}
      - O_2^\text{min} + s_9^2
    \right)
    + \lambda_{10}\left( \sum_{i \in \mathbb{K}_\ell}\beta_{3, i} y_{3, i}
      - O_3^\text{min} + s_{10}^2
    \right) \\
  &\qquad+ \lambda_{11}\left( \sum_{i \in \mathbb{K}_\ell}\beta_{1, i} y_{1, i}
      - O_1^\text{max} + s_{11}^2
    \right)
    + \lambda_{12}\left( \sum_{i \in \mathbb{K}_\ell}\beta_{2, i} y_{2, i}
      - O_2^\text{max} + s_{12}^2
    \right)
    + \lambda_{13}\left( \sum_{i \in \mathbb{K}_\ell}\beta_{3, i} y_{3, i}
      - O_3^\text{max} + s_{13}^2
    \right)  \\
  &\qquad+ \lambda_{14}\left(
      \sum_{\ell \in L} \sum_{i \in \mathbb{K}_\ell}\beta_{1, i} y_{1, i}
      - P^\text{min} + s_{14}^2
    \right)
    + \lambda_{15}\left( 
      \sum_{\ell \in  L} \sum_{i \in \mathbb{K}_\ell}\beta_{2, i} y_{2, i}
      - P^\text{min} + s_{15}^2
    \right)
    + \lambda_{16}\left( 
        \sum_{\ell \in L} \sum_{i \in \mathbb{K}_\ell}\beta_{3, i} y_{3, i}
      - P^\text{min} + s_{16}^2
    \right) \\
  &\qquad+ \lambda_{17}\left(
      \sum_{\ell \in L} \sum_{i \in \mathbb{K}_\ell}\beta_{1, i} y_{1, i}
      - P^\text{max} + s_{17}^2
    \right)
    + \lambda_{18}\left( 
      \sum_{\ell \in  L} \sum_{i \in \mathbb{K}_\ell}\beta_{2, i} y_{2, i}
      - P^\text{max} + s_{18}^2
    \right)
    + \lambda_{19}\left( 
        \sum_{\ell \in L} \sum_{i \in \mathbb{K}_\ell}\beta_{3, i} y_{3, i}
      - P^\text{max} + s_{19}^2
    \right) 
  + C
\end{align*}
$$

### Solve with one constraint `trade_num_upper.limit`

$$
\begin{align*}
\operatorname*{arg min}_{y_{\ell, i}} \,
  \frac{1}{2} \left(
  \sum_{\ell \in L} \sum_{i \in \mathbb{K}_\ell} \sum_{j \in \mathbb{K}_\ell}
     y_{\ell, i} \alpha_{\ell, i} \alpha_{\ell, j}  y_{\ell, j} \right) 
  - \sum_{\ell \in L} \sum_{i \in \mathbb{K}_\ell}
        (-\gamma_{\ell, i} - \lambda_1) y_{\ell, i}
    + \lambda_1\left( - M + s_1^2 \right)
\end{align*}
$$

use the transformation $y_{\ell, i} = \dfrac{1 - z_{\ell, i}}{2}$

$$
\begin{align*}
&= \operatorname*{arg min}_{z_{\ell, i} \in \{-1, 1\}} \,
  \sum_{\ell \in L} \sum_{i \in \mathbb{K}_\ell} \sum_{j \in \mathbb{K}_\ell \setminus \{i\}}
     \frac{\alpha_{\ell, i} \alpha_{\ell, j} }{4}z_{\ell, i}  z_{\ell, j} 
  - \sum_{\ell \in L} \sum_{i \in \mathbb{K}_\ell}
      \left(
        -\frac{1}{4}\sum_{j \in \mathbb{K}_{\ell}}
        \alpha_{\ell, i} \alpha_{\ell, j}
        + \frac{(-\gamma_{\ell, i} - \lambda_1)}{2}
      \right)z_{\ell, i} \\
  &\qquad + \frac{1}{8} \sum_{\ell \in L} \sum_{i \in \mathbb{K}_\ell} \sum_{j \in \mathbb{K}_\ell} 
     \alpha_{\ell, i} \alpha_{\ell, j}
    + \frac{1}{8} \sum_{\ell \in L} \sum_{i \in \mathbb{K}_\ell}  \alpha_{\ell, i}^2
    + \frac{1}{2} \sum_{\ell \in L} \sum_{i \in \mathbb{K}_\ell} (-\gamma_{\ell, i} - \lambda_1)
     + \lambda_1\left( - M + s_1^2 \right)
\end{align*}
$$

Load the data

In [2]:
with open("./data/1/31bonds/docplex-bin-avgonly-nocplexvars.json", 'r') as fp:
  bonds_data = json.load(fp) 

bonds_data

{'var': ['020002BJ9',
  '026874DS3',
  '081437AT2',
  '097023CJ2',
  '13645RAD6',
  '13645RBF0',
  '14448CBC7',
  '15135BAW1',
  '21871XAS8',
  '24422EWZ8',
  '24422EXP9',
  '314353AA1',
  '36166NAK9',
  '438516CM6',
  '443201AC2',
  '444859BR2',
  '444859BV3',
  '444859BY7',
  '444859CA8',
  '45687VAB2',
  '539830CD9',
  '540424AT5',
  '56501RAN6',
  '655844CR7',
  '655844CT3',
  '75513EAD3',
  '759351AP4',
  '760759BA7',
  '760759BC3',
  '907818FX1',
  '91324PEJ7'],
 'gamma': [-171.843480141259,
  -209.916098093165,
  -208.476006609091,
  -102.658455587925,
  -108.872687135303,
  -59.317668233044,
  -248.173707501463,
  -181.705222582624,
  -217.516328228335,
  -150.364227654476,
  -238.537754989087,
  -64.315591692518,
  -232.71668457355,
  -231.561240438637,
  -133.172276708478,
  -111.860973316693,
  -215.624770871618,
  -216.444892837478,
  -210.050748767208,
  -98.56277968679,
  -234.562738134276,
  -195.144598835887,
  -194.556369478498,
  -190.182474291791,
  -26.345955030805,

In [3]:
bonds_data["M"]

500.0

In [4]:
# Define parameters for the Hamiltonian
n = 31      # number of bonds
M = bonds_data["M"]
alpha_arr = bonds_data["alpha"]
gamma_arr = bonds_data["gamma"]
_lambda_arr = [1.]    # langrange multiplier
_slack_arr = [5.]      # slack variable

In [5]:
# Design the ansatz

def strongly_entangling_ansatz(params, num_of_qubits):
  """
  From PennyLane Codercise V.2.1 - Introduction to VQE
  """
  qml.StronglyEntanglingLayers(weights=params, wires=range(num_of_qubits))

In [6]:
# Define Hamiltonian
ZZ = [qml.PauliZ(i)@qml.PauliZ(j) for i in range(n) for j in range(i+1, n)]
ZZ_coeff = [0.25*alpha_arr[i]*alpha_arr[j] for i in range(n) for j in range(i+1, n)]

Z = [qml.PauliZ(i) for i in range(n)]
Z_coeff = [-0.25*sum(alpha_arr[i]*alpha_arr[j] for j in range(n)) 
  + 0.5*(-gamma_arr[i] - _lambda_arr[0]) for i in range(n)]

observables = ZZ + Z
coeffs = ZZ_coeff + Z_coeff
H = qml.Hamiltonian(coeffs, observables, grouping_type="qwc")

In [ ]:
dev = qml.device("lightning.gpu", wires=n)
# dev = qml.device("lightning.qubit", wires=n)
# dev = qml.device("default.qubit", wires=n)   # freezing


# Set the cost function on dev
@qml.qnode(dev, diff_method="parameter-shift")
def cost(x, n, H):
  strongly_entangling_ansatz(x, n)

  return qml.expval(H)

@qml.qnode(dev)
def probability_circuit(params, n):
  strongly_entangling_ansatz(params, n)

  return qml.probs(wires=range(n))

In [8]:
p = 2   # circuit repetition

seed = None
rng = np.random.default_rng(seed)
init_params = rng.random(size=(p, n, 3))

print(qml.draw(cost)(init_params, n, H))

 0: ──Rot(0.70,0.06,0.32)─╭●────────────────────────────────────────────────────────────────── ···
 1: ──Rot(0.17,0.56,0.72)─╰X─╭●─────────────────────────────────────────────────────────────── ···
 2: ──Rot(0.56,0.61,0.72)────╰X─╭●──────────────────────────────────────────────────────────── ···
 3: ──Rot(1.00,0.66,0.66)───────╰X─╭●───────────────────────────────────────────────────────── ···
 4: ──Rot(0.03,0.65,0.05)──────────╰X─╭●────────────────────────────────────────────────────── ···
 5: ──Rot(0.19,0.84,0.90)─────────────╰X─╭●─────────────────────────────────────────────────── ···
 6: ──Rot(0.89,0.53,0.40)────────────────╰X─╭●──────────────────────────────────────────────── ···
 7: ──Rot(0.26,0.65,0.09)───────────────────╰X─╭●───────────────────────────────────────────── ···
 8: ──Rot(0.63,0.13,0.26)──────────────────────╰X─╭●────────────────────────────────────────── ···
 9: ──Rot(0.37,0.96,0.77)─────────────────────────╰X─╭●─────────────────────────────────────── ···
10: ──Rot(

In [ ]:
p = 2   # circuit repetition

learning_rate = 0.03
opt = qml.QNGOptimizer(stepsize=learning_rate)

max_steps = 10
print_multiplier = 1

seed = None
rng = np.random.default_rng(seed)
params = rng.random(size=(p, n, 3))

old_cost = 9_999.
for i in range(max_steps):
  params = opt.step(cost, params, n, H)[0]
  obj_value = cost(params, n, H)

  if (i + 1) % print_multiplier == 0:
    print(f"Cost after step {i+1:5d}: {obj_value: .7f}")

    if np.round(old_cost, 7) == np.round(ob_value, 7):
      break
    else:
      old_cost = obj_value

print(f"Optimized parameters: {params}")
print(f"Optimized objective function value: {obj_value}")